## Notes : please use python version > 3.10 or 3.11 to run all the codes below 

# Installations

In [18]:
## import libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
from support_functions import missing_value, fill_missing, list_dtypes

## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle


ModuleNotFoundError: No module named 'support_functions'

In [14]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./sa-development.json"

In [16]:
project_id = 'dti-ds'
dataset_id = 'afif_dataset_036'
table_id = 'bank_marketing_campaign_forecast'
region = 'us-central1'
bucket_name = 'afif_gcs_036'

In [17]:
import os

model_name = "best_model.pkl"
local_model_dir = "models"
local_model_path = os.path.join(local_model_dir, model_name)

# Ensure the local directory exists
os.makedirs(local_model_dir, exist_ok=True)

try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob(f"model/{model_name}")
    blob_model.download_to_filename(local_model_path)

    print("Retrieve model succeeded")
except Exception as e:
    raise TypeError(f"An error occurred: {e}")

TypeError: An error occurred: File ./sa-development.json was not found.

# Model Prediction

In [12]:
import pickle
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from category_encoders import BinaryEncoder

# Define the model name and path
model_name = "best_model.pkl"
model_path = f"models/{model_name}"

# Load the saved model
loaded_model = pickle.load(open(model_path, 'rb'))

# Sample data for prediction (adapted to match your dataset features)
test_data = [
    [55, 'admin.', 1662, 'no', 'no', 'cellular', 'jun', 2, -1, 'unknown', '56+'],
    [39, 'self-employed', -3058, 'yes', 'yes', 'cellular', 'apr', 3, -1, 'unknown', '36-45']
]

# Column names based on your dataset
columns = ['age', 'job', 'balance', 'housing', 'loan', 'contact', 'month', 
           'campaign', 'pdays', 'poutcome', 'age_range']

# Create DataFrame
df = pd.DataFrame(test_data, columns=columns)

# Ensure that all categorical variables are encoded as necessary (if required by the model)
# Assuming your model was trained with BinaryEncoder for categorical variables
# Adjust the following transformations as per your preprocessing steps
encoder = BinaryEncoder(cols=['job', 'housing', 'loan', 'contact', 'month', 'poutcome', 'age_range'])
df_encoded = encoder.fit_transform(df)

# Predict using the loaded model
predictions = loaded_model.predict(df_encoded)

# Print predictions
print(predictions)


ModuleNotFoundError: No module named 'category_encoders'

In [ ]:
# load data from BQ
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
test_df = query_job.result().to_dataframe()

test_df

In [ ]:
bulk_predict_df = test_df

In [ ]:
y_pred = loaded_model.predict(bulk_predict_df)
y_pred_df = pd.DataFrame(y_pred, columns=['SubscriptionPrediction'])


result_df = pd.concat([bulk_predict_df.reset_index(drop=True), y_pred_df.reset_index(drop=True)], axis=1)
result_df